In [6]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [12]:

# Read the data
data = pd.read_csv('./melb_data.csv')

# Separate target from predictors
y = data.Price
X = data.drop(['Price'], axis=1)

# Divide data into training and validation subsets
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)

categorical_cols = [col for col in X_train_full.columns if X_train_full[col].nunique() < 10 and X_train_full[col].dtype == 'object' ]

# Select numerical columns
numerical_cols = [cname for cname in X_train_full.columns if X_train_full[cname].dtype in ['int64', 'float64']]

# Keep selected columns only
my_cols = categorical_cols + numerical_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()

In [13]:
numerical_transformer = SimpleImputer(strategy='constant')

categorical_transformer = Pipeline( steps = [
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer([
    ('num', numerical_transformer, numerical_cols), # type: ignore
    ('cat', categorical_transformer, categorical_cols) # type: ignore
])

In [14]:
model = RandomForestRegressor(n_estimators= 25, random_state= 0)

In [15]:
my_pipeline = Pipeline(steps = [
    ('preprocessor', preprocessor),
    ('model', model)
])
my_pipeline.fit(X_train, y_train)
preds = my_pipeline.predict(X_valid)

score = mean_absolute_error(y_valid, preds)

In [17]:
print(f"Score is: {score}")

Score is: 164329.71709516796


In [20]:
output = pd.DataFrame({'Id': X_valid.index,
                       'SalePrice': preds,
                       'TruePrice': y_valid})
output.to_csv('submission.csv', index=False)